# Sample code for optimzation schemes

This script can be used to replicate the results in Fig. 2 a) and b) in the publication

M. Franckie and J. Faist, 2020 (submitted manuscript)

## Prerequiests

The following python packages need to be installed:

    aftershoq    : https://github.com/mfranckie/aftershoq (pip install aftershoq)
    GPyOpt       : https://github.com/SheffieldML/GPyOpt  (pip install
    HilbertCurve : https://github.com/galtay/hilbertcurve (automatically installed with aftershoq above)
    numpy
    scipy
    matplotlib
    

## Instructions

The variable "gen" switches between the different data sets:

    gen = 1: 2-well without e-e scattering
    gen = 2: 2-well with e-e scattering
    gen = 3: 3-well without e-e scattering
    
The Gaussian process models which have been fitted to those data sets are then loaded, and used as sample functions in subsequent optimization.

The "optimizer" variable selects the optimization strategy:

    optimizer = 'Gauss' : One-dimensional Gaussian Process regression
    optimizer = 'MDGP'  : Multi-dimensional Gaussian Process regression
    optimizer = 'GA'    : Genetic algorithm
    optimizer = 'IAPT'  : Information algorithm with parallel trials
    optimizer = 'RANDOM': Random selection of samples


The variable "p" is the Hilbert curve parameter, and "r" is the IAPT parameter. The optimization parameters are

    Ngen : the number of generations to optimize for
    N0   : the number of samples in each generation

In [ ]:
from aftershoq.numerics import GAopt, Paraopt, Gaussopt
from aftershoq.utils.hilbertutil import HilbertUtil
from hilbert_curve.hilbert import HilbertCurve
import GPy
import GPyOpt
import numpy as np
from matplotlib import pyplot as plt
import copy
import random
import pickle
import io

In [ ]:
# Build limits and load pickled models

plots = True

gen = 3

p = 8

r = 1.1

Ngen = 70
N0 = 10

#optimizer = 'Gauss' # One-dimensional Gaussian Process regression
optimizer = 'MDGP' # Multi-dimensional Gaussian Process regression
#optimizer = 'GA'  # Genetic algorithm
#optimizer = 'IAPT' # Information algorithm with parallel trials
#optimizer = 'RANDOM' # Random selection of samples

print(f'Optimization scheme: {optimizer}')

frac_points = 1

if gen == 2:
    print('Loading 2-well Gen II data')
    names = ['well1','barrier1','well2L','well2R','barrier2','x']
    paramindex = [2,3,4,6,7,11]
    orig = np.array([8.28,1.97,8.7,5.35,3.38,0.25])
    yindex = 1
    model_name = 'trained_models/GPmodel2well'
elif gen == 1:
    print('Loading 2-well Gen I data')
    path = '../data/results_testpipe.log'
    names = ['well1','barrier1','well2','barrier2']
    paramindex = [1,2,3,4]
    orig = np.array([17.7,3.1,8.5,1.8])
    expect_length = 8
    yindex = 7
    model_name = 'trained_models/GPmodel2well_no-e-e'
elif gen == 3:
    print('Loading 3-well data')
    path = '../data/results_3well_all.log'
    names = ['barr1','well1','barr2','well2','barr3','well3L','well3R','x']
    paramindex = [2,3,4,5,6,7,9,13]
    expect_length = 16
    yindex = 1
    orig = np.array([4.3, 8.9, 2.46, 8.15, 4.1, 5.5, 5.5, 0.25])
    model_name = 'trained_models/GPmodel3well'

bo_step = pickle.load(open(model_name,'rb'))

limits = []
names = []
for i in range(len(paramindex)):
    limits.append(bo_step.domain[i]['domain'])
    names.append(bo_step.domain[i]['name'])

print(f'The domain for each parameter: \n{names}\n{limits}')

print('\nThe hyperparameters are:')
print(bo_step.model.get_model_parameters_names())
print(bo_step.model.get_model_parameters())

In [ ]:
# plot plane with first two variables

i1 = 1
i2 = 3

Np = 50

x1 = np.linspace(limits[i1][0],limits[i1][1],Np)
x2 = np.linspace(limits[i2][0],limits[i2][1],Np)

params = copy.deepcopy(orig)

mean = []
std = []

for xx1 in x1:
    meanrow = []
    stdrow = []
    params[i1] = xx1
    for xx2 in x2:
        params[i2] = xx2
        value = bo_step.model.predict(params)
        meanrow.append(value[0][0])
        stdrow.append(value[1][0])
    mean.append(meanrow)
    std.append(stdrow)

mean = np.squeeze(mean)
std = np.squeeze(std)

if plots:
    plt.figure('Mean')
    plt.imshow(np.transpose(mean), origin='lower', extent=(limits[i1][0],limits[i1][1],limits[i2][0],limits[i2][1]),
               aspect='auto',cmap='gnuplot')
    plt.xlabel(names[i1])
    plt.ylabel(names[i2])
    plt.colorbar(label='Posterior mean (1/cm)')
    plt.scatter(orig[i1],orig[i2],marker='*',s=100,c='black',label='original')
    plt.legend()

    plt.figure('Std')
    plt.imshow(np.transpose(std), origin='lower', extent=(limits[i1][0],limits[i1][1],limits[i2][0],limits[i2][1]),
               aspect='auto',cmap='gnuplot')
    plt.xlabel(names[i1])
    plt.ylabel(names[i2])
    plt.colorbar(label='Posterior std (1/cm)')
    plt.scatter(orig[i1],orig[i2],marker='*',s=100,c='black',label='original')
    plt.legend()
    plt.show()


In [ ]:
# ## Optimizing on the GP
#
# Testing opitmization for different schemes.

pathsave = "."


def predict(x):
    y = bo_step.model.predict(np.array(x))
    return -np.squeeze(y[0]).tolist()

def plotGP(model,Np,i1,i2,x0,limits):
    x = np.linspace(limits[i1][0],limits[i1][1],Np)
    y = np.linspace(limits[i2][0],limits[i2][1],Np)
    mean = []
    std = []
    for xx in x:
        rowstd = []
        rowmean = []
        x0[i1] = xx
        for yy in y:
            x0[i2] = yy
            val = model.predict(x0)
            rowstd.append(val[1])
            rowmean.append(val[0])
        mean.append(rowmean)
        std.append(rowstd)
    mean = np.squeeze(mean)
    std = np.squeeze(std)
    fig,(ax1,ax2) = plt.subplots(1,2,num='GP',clear=True)
    ax1.imshow(np.transpose(mean), origin='lower', extent=(limits[i1][0],limits[i1][1],limits[i2][0],limits[i2][1]),
           aspect='auto',cmap='gnuplot')

    ax2.imshow(np.transpose(std), origin='lower', extent=(limits[i1][0],limits[i1][1],limits[i2][0],limits[i2][1]),
           aspect='auto',cmap='gnuplot')
    plt.pause(0.001)


dpar, parmin = [], []
[dpar.append((limits[i][1]-limits[i][0])/2) for i in range(len(limits))]
[parmin.append(limits[i][0]) for i in range(len(limits))]
dpar=np.array(dpar)
parmin=np.array(parmin)




ND = np.shape(limits)[0]
print(ND)

x0, y0 = [], []

for _ in range(N0):
    xi = []
    for i in range(ND):
        xi.append( random.random()*(limits[i][1]-limits[i][0]) + limits[i][0] )
    xi = np.array(xi)
    x0.append(xi)
    y0.append(predict(xi))
print(f'Initial merit values = {y0}')

hc = HilbertCurve( p, ND )
hu = HilbertUtil( hc )
xhil, yhil = [], []
d = 0
xhil.append(d)
x = hu.interp_coords_from_dist(d)
x = np.squeeze(hu.scale_coordinates(x, dpar, parmin))
yhil.append( predict(x) )
for _ in range(N0-2):
    d = random.random()*hu.imax
    xhil.append(d)
    x = hu.interp_coords_from_dist(d)
    x = np.squeeze(hu.scale_coordinates(x, dpar, parmin))
    yhil.append( predict( x ) )
d = hu.imax
xhil.append(d)
x = hu.interp_coords_from_dist(d)
x = np.squeeze(hu.scale_coordinates(x, dpar, parmin))
yhil.append( predict(x) )

if ( optimizer == 'GA'):
    opt = GAopt(tolerance=0.1, maxiter=10, population=N0, x0 = x0, y0 = y0,
                mutrate = 0.50, mutsize=0.5, corate = 0.5, limits = limits,
                nparents = 2, noff = 3)
elif( optimizer =='RANDOM'):
    opt = GAopt(tolerance=0.1, maxiter=10, population=N0, x0 = x0, y0 = y0,
                mutrate = 1.0, mutsize=1.0, corate = 2.0, limits = limits,
                nparents = 2, noff = 3)
elif (optimizer == 'IAPT'):
    opt = Paraopt(tolerance=0.1, r=r, maxiter=10, procmax=N0, x0 = xhil, y0 = yhil)
elif (optimizer == 'Gauss'):
    opt = Gaussopt(tolerance=0.00*hu.imax, maxiter=10, procmax=N0, x0 = xhil, y0 = yhil,
                   sigma=5, l = 5, padding = 0.005*hu.imax,
                   sigma_noise = 5, sigma_noise_max =5)



ID = random.randint(0,10000)

title = f"{optimizer} ND={ND} N0={N0} ID={ID}"
if optimizer == 'Gauss':
    title += f"\n[sigma, l, sigma_noise]={opt.theta0}"
elif optimizer == 'IAPT':
    title += f"\nr={opt.r}"

if plots:
    plt.figure(1)
    plt.clf()
    plt.title(title)
    cmin, cmax = -4.5, -1
    plt.imshow(np.transpose(mean), origin='lower', extent=(limits[i1][0],limits[i1][1],limits[i2][0],limits[i2][1]),
           aspect='auto',cmap='gnuplot')
    plt.colorbar()
    #model.plt(np.linspace(lim_min,lim_max, 200),np.linspace(lim_min,lim_max, 200))
              #cmin=cmin, cmax=cmax)


if optimizer == 'Gauss' or optimizer == 'IAPT':
    x = []
    for xx in opt.x:
        x.append(np.squeeze(hu.scale_coordinates(hu.interp_coords_from_dist(xx), dpar, parmin)))
    x = np.transpose(np.array(x))
    y = np.squeeze(opt.y)
    if plots:
        plt.figure(5)
        d = np.linspace(0, hu.imax, min(hu.imax,5000))
        h = [predict(np.squeeze(hu.scale_coordinates(hu.interp_coords_from_dist(dd), dpar, parmin))) for dd in d]
        plt.plot(d, h, 'r-')
elif optimizer == 'MDGP':
    x = np.transpose(np.array(x0))
    y = np.transpose(np.array(y0))

else:
    x = np.transpose(np.array(opt.x))
    y = np.transpose(np.array(opt.y))
#print(y)
if plots:
    plt.figure(1)
    plt.scatter(x[i1], x[i2], c=y.ravel().tolist(), marker = '+', cmap='hot')
    #pl.clim((cmin,cmax))
    #pl.colorbar()
    plt.figure(2)
    plt.title(title)

ymin = []
gen = []

#if optimizer == 'MDGP':
#    x = np.transpose(x)

filename = f"{pathsave}/{optimizer}_{ID}"
if optimizer == 'Gauss':
    #filename += f"_{}"
    pass
elif optimizer == 'IAPT':
    filename += f'r={opt.r}p={p}'
filename +=".dat"

for g in range(Ngen):

    if optimizer == 'GA' or optimizer == 'RANDOM':
        oldx = opt.x
        newx = opt.nextstep()
        newy = []
        [newy.append( predict(x)) for x in newx]
        opt.addpoints(newx, newy)

        x = np.transpose(np.array(opt.x))
        y = np.array(opt.y)
    elif optimizer == 'MDGP':
        y = np.reshape(y,(len(y),1))
        x = np.transpose(x)

        GP_it = GPyOpt.methods.BayesianOptimization(f = None, domain = bo_step.domain, X = x, Y = y,
                                                 acquisition_type='EI',normalize_Y=False,batch_size=N0,num_cores=N0,
                                                   maximize=False,evaluator_type='local_penalization')
        x = np.transpose(x)
        newx = GP_it.suggest_next_locations()

        newy = []
        [newy.append( predict(x) ) for x in newx]

        for xx in newx:
            x = np.append( np.transpose(x), xx )

        y = np.append(y,newy)
        x = np.reshape(x,(len(y),ND))
        x = np.transpose(x)

        if plots: plotGP(GP_it.model,20,i1,i2,orig,limits)

    else:
        newx = opt.nextstep()
        newy = []
        x = []

        for xx in newx:
            xx = hu.interp_coords_from_dist(xx)
            xx=np.squeeze(hu.scale_coordinates(np.array(xx),dpar, parmin) )
            newy.append( predict(xx) )

        opt.addpoints(newx,newy)

        x = []
        for xx in opt.x:
            xx = hu.interp_coords_from_dist(xx)
            x.append(hu.scale_coordinates(xx, dpar, parmin))
        x = np.transpose(np.squeeze( np.array(x) ) )
        y = np.squeeze(opt.y)

    #print(x)
    ymin.append(np.min(y))
    gen.append(g)

    if plots:
        #plt.clf()
        plt.figure(1)
        if ND == 2:
            plt.plot(x[i1], x[i2], 'g+')
        else:
            plt.scatter(x[i1], x[i2], c=y, marker = '+', cmap='hot')
            #pl.clim((cmin,cmax))
        plt.pause(0.01)
        #pl.xlim((-1,2))
        #pl.ylim((-1,2))

        plt.figure(2)
        plt.plot(gen, ymin, 'k')

        plt.pause(0.01)

        if optimizer == 'Gauss':
            opt.plotGP()
            plt.plot(d, h, 'r--')
            #pl.plot(opt.x, opt.y, '+', opt.xt, opt.mean)
            #pl.fill_between(opt.xt, opt.mean-opt.theta[0], opt.mean+opt.theta[0])
            plt.pause(0.01)
        if optimizer == 'IAPT':
            plt.figure(5)
            plt.plot(opt.x, opt.y, '+')
            plt.pause(0.01)
            
    np.savetxt(filename, ymin)

best_ind = np.argmin(y)
x = np.transpose(x)
print(f'Optimal value found = {y[best_ind]}\n at {x[best_ind]}')


if optimizer == 'MDGP' and plots:
    plotGP(GP_it.model, 50, i1, i2, x[best_ind], limits)

if plots:
    plt.show()